In [12]:
# Getting dependencies
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time
import threading

In [17]:
df_vgchart = pd.read_csv('Data/videogamechart.csv')
df_vgchart = df_vgchart[df_vgchart['console']=='PC']
df_vgchart

,Unnamed: 0,img,title,console,genre,publisher,developer,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
11,14,/games/boxart/full_9031276AmericaFrontccc.jpg,Grand Theft Auto V,PC,Action,Rockstar Games,Rockstar North,NaN,NaN,NaN,12.60,NaN,NaN,NaN,NaN,NaN,2015-04-14,2018-11-02
21,24,/games/boxart/full_the-binding-of-isaac_366Ame...,The Binding of Isaac,PC,Action,Edmund McMillen,Edmund McMillen,NaN,8.3,NaN,7.17,NaN,NaN,NaN,NaN,NaN,2011-09-28,2018-12-01
35,38,/games/boxart/full_5321723AmericaFrontccc.jpg,Tomb Raider (2013),PC,Action,Square Enix,Crystal Dynamics,NaN,8.8,NaN,5.50,NaN,NaN,NaN,NaN,NaN,2013-03-05,2018-01-06
48,51,/games/boxart/full_dead-island_410AmericaFront...,Dead Island,PC,Action,Deep Silver,Techland,NaN,7.1,NaN,4.52,NaN,NaN,NaN,NaN,NaN,2011-09-06,NaN
56,59,/games/boxart/full_8931458AmericaFrontccc.jpg,7 Days to Die,PC,Action,The Fun Pimps,The Fun Pimps,NaN,NaN,NaN,4.18,NaN,NaN,NaN,NaN,NaN,2013-12-13,2018-05-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57897,58803,/games/boxart/full_7024289AmericaFrontccc.jpg,WILL: A Wonderful World,PC,Visual Novel,AGM Playism,4D Door Games,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-06-06,2018-12-17
57898,58804,/games/boxart/default.jpg,Without Within,PC,Visual Novel,InvertMouse,InvertMouse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-22,2018-12-25
57899,58805,/games/boxart/default.jpg,Without Within 2,PC,Visual Novel,InvertMouse,InvertMouse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-09,2018-12-25
57900,58806,/games/boxart/default.jpg,Without Within 3,PC,Visual Novel,InvertMouse,InvertMouse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-03,2018-12-25


In [18]:
df_vgchart = df_vgchart[['title', 'genre' ,  'total_sales', 'na_sales', 'release_date']]
df_vgchart.head(50)

,title,genre,total_sales,na_sales,release_date
11,Grand Theft Auto V,Action,NaN,NaN,2015-04-14
21,The Binding of Isaac,Action,NaN,NaN,2011-09-28
35,Tomb Raider (2013),Action,NaN,NaN,2013-03-05
48,Dead Island,Action,NaN,NaN,2011-09-06
56,7 Days to Die,Action,NaN,NaN,2013-12-13
57,Just Cause 2,Action,NaN,NaN,2010-03-23
67,Diablo III: Reaper of Souls,Action,NaN,NaN,2014-03-25
79,Dying Light,Action,NaN,NaN,2016-02-06
100,Hitman: Absolution,Action,NaN,NaN,2012-11-20
104,The Binding of Isaac: Rebirth,Action,NaN,NaN,2014-11-04


In [19]:
df_vgchart = df_vgchart.dropna()
df_vgchart.head(50)

,title,genre,total_sales,na_sales,release_date
344,Tomb Raider,Action,1.03,0.96,1996-11-14
369,Grand Theft Auto: San Andreas,Action,0.97,0.00,2005-06-07
404,Grand Theft Auto IV,Action,0.87,0.01,2008-12-02
460,Assassin's Creed: Revelations,Action,0.76,0.16,2011-11-29
572,Assassin's Creed: Brotherhood,Action,0.59,0.16,2011-03-22
612,Assassin's Creed: Unity,Action,0.56,0.20,2014-11-11
630,Assassin's Creed II,Action,0.54,0.01,2010-03-09
663,Dishonored,Action,0.51,0.27,2013-04-16
821,Tomb Raider: The Last Revelation,Action,0.41,0.41,1999-10-31
892,Saints Row: The Third,Action,0.38,0.16,2011-11-15


In [20]:
df_vgchart = df_vgchart.sort_values(by = 'total_sales', ascending = False)
df_vgchart.head()

,title,genre,total_sales,na_sales,release_date
45884,The Sims 3,Simulation,7.96,1.01,2009-06-02
45890,Microsoft Flight Simulator,Simulation,5.12,3.22,1996-10-31
45894,The Sims 4,Simulation,4.10,1.20,2014-09-02
36232,The Elder Scrolls V: Skyrim,Role-Playing,3.99,1.18,2011-11-11
45896,The Sims: Unleashed,Simulation,3.76,2.03,2002-09-23


In [29]:
df_vgchart.sort_values(by = 'release_date', ascending = False)
df_vgchart.head

<bound method NDFrame.head of                                         title         genre  total_sales  \
45884                              The Sims 3    Simulation         7.96   
45890              Microsoft Flight Simulator    Simulation         5.12   
45894                              The Sims 4    Simulation         4.10   
36232             The Elder Scrolls V: Skyrim  Role-Playing         3.99   
45896                     The Sims: Unleashed    Simulation         3.76   
...                                       ...           ...          ...   
27322         The SpongeBob SquarePants Movie      Platform         0.00   
27364  Disney Epic Mickey 2: The Power of Two      Platform         0.00   
30589                            Tumblebugs 2        Puzzle         0.00   
34549            Superstars V8 Next Challenge        Racing         0.00   
55058                 Elemental: War of Magic      Strategy         0.00   

       na_sales release_date  
45884      1.01   2009-06-

In [30]:
df_vgchart.to_csv('Data/clean_vg_chart.csv', index = False)